In [6]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from ema_workbench import (
    ScalarOutcome,
    load_results,
    Policy,
    Scenario,
    MultiprocessingEvaluator
)

from ema_workbench import load_results
from ema_workbench.analysis import prim
from custom_problem_formulation_no_RfR import get_model_for_problem_formulation

In [10]:
combined_df = pd.read_csv("results/PRIM_scenarios.csv", index_col=0)
combined_df

,A.0_ID flood wave shape,A.1_Bmax,A.1_Brate,A.1_pfail,A.2_Bmax,A.2_Brate,A.2_pfail,A.3_Bmax,A.3_Brate,A.3_pfail,...,A.4_DikeIncrease 0,A.4_DikeIncrease 1,A.4_DikeIncrease 2,A.5_DikeIncrease 0,A.5_DikeIncrease 1,A.5_DikeIncrease 2,scenario,policy,model,composite_ooi
5666,104,40.525349,10.0,0.161001,187.789168,1.5,0.010373,205.979011,1.0,0.958783,...,0,0,0,0,0,0,5666,No Policy,dikesnet,3.851408
37006,112,45.942740,10.0,0.048710,140.411132,1.0,0.024664,82.938927,1.5,0.397429,...,0,0,0,0,0,0,37006,No Policy,dikesnet,3.842766
10872,124,40.037075,1.5,0.142803,341.756695,1.5,0.036182,77.097873,1.5,0.527626,...,0,0,0,0,0,0,10872,No Policy,dikesnet,3.806186
23514,8,71.302063,1.0,0.046825,345.456662,1.0,0.011393,30.957752,1.5,0.700170,...,0,0,0,0,0,0,23514,No Policy,dikesnet,3.743752
456,123,43.660903,10.0,0.199587,181.602356,1.0,0.058039,222.809776,1.5,0.212183,...,0,0,0,0,0,0,456,No Policy,dikesnet,3.716054
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7221,94,331.168259,1.5,0.348061,289.915138,1.5,0.735999,188.403412,10.0,0.225255,...,0,0,0,0,0,0,7221,No Policy,dikesnet,1.223055
29071,9,304.130009,10.0,0.346508,183.835119,1.0,0.674772,122.234986,10.0,0.174207,...,0,0,0,0,0,0,29071,No Policy,dikesnet,1.200517
3619,51,82.911645,1.0,0.345628,305.490519,10.0,0.946479,135.541766,10.0,0.504624,...,0,0,0,0,0,0,3619,No Policy,dikesnet,1.198221
8470,51,248.465616,10.0,0.341540,340.138214,10.0,0.313503,81.810139,1.0,0.344787,...,0,0,0,0,0,0,8470,No Policy,dikesnet,1.139769


In [13]:
# First we test the final policy set, against each scenario, which remained after the PRIM analysis.

model, steps = get_model_for_problem_formulation()

final_policies = pd.read_csv('results/levers_final_policies.csv')
final_policies = final_policies[[l.name for l in model.levers]]

policies = []
for idx, row in final_policies.iterrows():
    policy_dict = row.to_dict()
    policy_name = idx
    policies.append(Policy(policy_name, **policy_dict))

# combined_df = pd.read_csv('results/combined_df.csv')    # NOG genereren
scenarios_df = combined_df[[u.name for u in model.uncertainties]]

scenarios = []
for idx, row in scenarios_df.iterrows():
    scenario_dict = row.to_dict()
    scenario_name = idx
    scenarios.append(Scenario(scenario_name, **scenario_dict))


In [14]:
n_scenarios =  50

with MultiprocessingEvaluator(model) as evaluator:
            results = evaluator.perform_experiments(n_scenarios, policies)

100%|████████████████████████████████████████| 200/200 [00:32<00:00,  6.08it/s]
